<a href="https://colab.research.google.com/github/noemiezufferey/2024_Geoinformatique_I/blob/main/s4_ip_exercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pooch
import pooch
import zipfile
import os

# Définir l'URL et le chemin de destination
url = "https://files.swisscarto.ch/geoinf1/vd_dgav_surfaces_agricoles.zip"
destination = pooch.os_cache("geodata")  # Crée un dossier de cache pour stocker les fichiers

# Télécharger l'archive
filename = pooch.retrieve(url, known_hash=None, path=destination)

# Décompresser l'archive ZIP
with zipfile.ZipFile(filename, 'r') as zip_ref:
    # Extraire tous les fichiers dans le même répertoire que l'archive
    zip_ref.extractall(destination)

print(f"Les fichiers ont été extraits dans : {destination}")




SHA256 hash of downloaded file: 0d866067c647f235d92499e669b8dfb4dfea18da3cfd99434a7e6176a9fba81f
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


Les fichiers ont été extraits dans : /root/.cache/geodata


In [ ]:
from google.colab import files

# Permet de télécharger un fichier depuis votre ordinateur
uploaded = files.upload()

# Afficher le nom des fichiers téléchargés
uploaded.keys()


Saving perimetre_ze_joint.gpkg to perimetre_ze_joint.gpkg


dict_keys(['perimetre_ze_joint.gpkg'])

In [ ]:
!pip install geopandas
from google.colab import files

# Télécharger le fichier depuis votre machine locale
uploaded = files.upload()

import geopandas as gpd

# Lire le fichier GeoPackage avec GeoPandas
gdf = gpd.read_file("vd_dgav_surfaces_agricoles/nom_du_fichier.gpkg", layer="agr_dgav_sau")

# Afficher les 5 premières lignes du DataFrame
gdf.head()



In [ ]:
import geopandas as gpd
import pandas as pd
import pooch
import os
from google.colab import files

# Télécharger et décompresser les données
url = 'https://files.swisscarto.ch/geoinf1/vd_dgav_surfaces_agricoles.zip'
data_files = pooch.retrieve(
    url=url,
    known_hash=None,
    processor=pooch.Unzip()
)

# Lire le fichier Geopackage avec les surfaces agricoles
sau_gpkg_path = [df for df in data_files if df.endswith('.gpkg')][0]
sau = gpd.read_file(sau_gpkg_path, layer='agr_dgav_sau')

# Télécharger le fichier de périmètre d'évaluation
uploaded = files.upload()

perimetre_evaluation_path = list(uploaded.keys())[0]
perimetre_evaluation = gpd.read_file(perimetre_evaluation_path)

from google.colab import files
uploaded = files.upload()  # Uploader le fichier .shp et ses fichiers connexes

# Vérifier les fichiers uploadés
print(uploaded.keys())

# Extraire le fichier shapefile s'il est compressé dans une archive .zip
import zipfile
with zipfile.ZipFile("Zone_de_risque.zip", "r") as zip_ref:
    zip_ref.extractall("temp_shapefile")

# Lire le fichier shapefile extrait
perimetre_evaluation = gpd.read_file("temp_shapefile/Zone_de_risque.shp")


# Vérifier les systèmes de coordonnées et les harmoniser si nécessaire
if sau.crs != perimetre_evaluation.crs:
    perimetre_evaluation = perimetre_evaluation.to_crs(sau.crs)

# Faire une extraction des polygones des surfaces agricoles utiles qui intersectent avec le périmètre d'évaluation
sau_intersect = gpd.overlay(sau, perimetre_evaluation, how='intersection')

# Vérifier le résultat en l'affichant sur une carte
sau_intersect.plot()

# Lire le fichier CSV avec les informations d'utilisation
csv_path = [df for df in data_files if df.endswith('.csv')][0]
sau_util = pd.read_csv(csv_path)

# Afficher les premières lignes des deux jeux de données pour identifier les colonnes correspondantes
print(sau_intersect.head())
print(sau_util.head())

# Faire une jointure par attributs pour rajouter les informations du fichier CSV dans le DataFrame de GeoPandas
sau_data = sau_intersect.merge(sau_util, left_on='ident', right_on='IDENT', how='left')

# Calculer la superficie de chaque surface agricole en hectares
sau_data['superficie_ha'] = sau_data['geometry'].area / 10000

# Faire un résumé des surfaces totales par utilisation
sau_area_par_classe = sau_data.groupby('UTILISATIO')['superficie_ha'].sum().reset_index()

# Sauvegarder le résumé des surfaces totales par utilisation dans un fichier CSV
sau_area_par_classe.to_csv('resume_surfaces_agricoles.csv', index=False)

# Supprimer la colonne 'IDENT' pour éviter les conflits de noms de colonnes
del sau_data['IDENT']

# Sauvegarder la version modifiée du DataFrame GeoPandas avec les surfaces agricoles dans un fichier Geopackage
sau_data.to_file('surfaces_agricoles_utilisation.gpkg', driver='GPKG', layer='surfaces_agricoles_utilisation')

# Télécharger les fichiers depuis Google Colab
files.download('resume_surfaces_agricoles.csv')
files.download('surfaces_agricoles_utilisation.gpkg')



Saving perimetre_evaluation.shp to perimetre_evaluation.shp


DataSourceError: Unable to open perimetre_evaluation.shx or perimetre_evaluation.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.